In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install openai

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
from datasets import load_dataset
import pandas as pd
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('openai')
client = OpenAI(api_key=api_key)

In [ ]:

# yahoo_answers_topics
# yahoo_answers_topics
# has 10 topics, could maybe take 20 from each topic and rephrase them all.
# take question_content, question_title and best_answer (to avoid bias towards questions or answers)
# Load the dataset
dataset = load_dataset("yahoo_answers_topics")

# Process each topic
yahoo_processed_data = []
yahoo_classes = ["Society & Culture", "Science & Mathematics", "Health", "Education & Reference", "Computers & Internet", "Sports", "Business & Finance", "Entertainment & Music", "Family & Relationships", "Politics & Government"]

for topic in range(10):  # There are 10 topics
    # Filter by topic and take 20 samples
    topic_data = dataset['train'].filter(lambda example: example['topic'] == topic).select(range(20))

    for item in topic_data:
        # Combine question_title and question_content for the question
        question_text = f"{item['question_title']} {item['question_content']}"
        yahoo_processed_data.append({
            "description": f"{yahoo_classes[topic]}",
            "text": question_text
        })

        # Add best_answer as a separate row
        yahoo_processed_data.append({
            "description": f"{yahoo_classes[topic]}",
            "text": item['best_answer']
        })


In [ ]:
# https://huggingface.co/datasets/cardiffnlp/tweet_topic_multi/viewer/tweet_topic_multi/test_2020?p=2&row=250
# take text and label name, and split array to 1 per line

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_topic_multi")

# Initialize a list to hold the processed data
tweet_processed_data = []

# Define the splits to process
splits_to_process = ['test_2021', 'test_2020']

for split in splits_to_process:
    # Iterate through each entry in the split
    for item in dataset[split]:
        # Extract the first value of label_name array as description
        description = item['label_name'][0] if item['label_name'] else 'No Label'

        # Add the processed data
        tweet_processed_data.append({
            "description": description,
            "text": item['text']
        })

In [ ]:
# https://huggingface.co/datasets/KnutJaegersberg/wikipedia_categories/viewer/default/train?p=4
# not sure about this one because it's only keywords, maybe include a few for robustness

# Load the dataset
dataset = load_dataset("KnutJaegersberg/wikipedia_categories")

# Initialize a list to hold the processed data
wiki_processed_data = []

# Iterate through each entry in the dataset
for item in dataset['train']:  # Assuming we are processing the 'train' split
    # Use the 'c4' column as description
    description = item['c4']

    # Add the processed data
    wiki_processed_data.append({
        "description": description,
        "text": item['text']
    })

In [ ]:

# https://huggingface.co/datasets/valurank/News_Articles_Categorization/viewer/default/train?p=2
# 8 categories, so take a few per cat, and data augement categories
# issues because full articles so maybe it's too long

In [ ]:

# https://huggingface.co/datasets/HuggingFaceH4/stack-exchange-preferences
# this one seems nice but a lot of data, would need to filter to see how many different exchanges there are and take a few per exchange
# 80GB of data, so removed for now because crashes low ram machine
"""
import re

# Load the dataset
dataset = load_dataset("HuggingFaceH4/stack-exchange-preferences")

# Initialize a list to hold the processed data
processed_data = []

# Function to extract the topic from URL
def extract_topic(url):
    match = re.search(r'https://([a-zA-Z0-9]+)\.stackexchange\.com', url)
    return match.group(1) if match else 'Unknown'

# Process the dataset
for item in dataset['train']:  # Assuming we are processing the 'train' split
    # Extract topics from metadata
    topics = [extract_topic(url) for url in item['metadata'] if 'stackexchange.com' in url]

    # Use the first topic as the primary topic for this entry
    primary_topic = topics[0] if topics else 'Unknown'

    # Add to processed data
    processed_data.append({
        "qid": item['qid'],
        "question": item['question'],
        "answers": item['answers'],
        "date": item['date'],
        "metadata": item['metadata'],
        "topic": primary_topic
    })

# Analyze the unique topics
unique_topics = set(entry['topic'] for entry in processed_data)
print("Unique Topics Count:", len(unique_topics))
print("Unique Topics:", unique_topics)


from html import unescape
from bs4 import BeautifulSoup
import random

# finish processing the stackexchange data

# Function to convert HTML to plain text
def html_to_text(html):
    soup = BeautifulSoup(unescape(html), "html.parser")
    return soup.get_text(separator=' ')

# Convert HTML to plain text for each question
for entry in processed_data:
    entry['text'] = html_to_text(entry['question'])

# Group data by topic and sample 30 datapoints per topic
grouped_data = {}
for entry in processed_data:
    topic = entry['topic']
    if topic not in grouped_data:
        grouped_data[topic] = []
    grouped_data[topic].append(entry)

sampled_data = []
for topic, entries in grouped_data.items():
    if len(entries) > 30:
        sampled_entries = random.sample(entries, 30)
    else:
        sampled_entries = entries
    for entry in sampled_entries:
        sampled_data.append({
            "description": topic,
            "text": entry['text']
        })

  """


'\nimport re\n\n# Load the dataset\ndataset = load_dataset("HuggingFaceH4/stack-exchange-preferences")\n\n# Initialize a list to hold the processed data\nprocessed_data = []\n\n# Function to extract the topic from URL\ndef extract_topic(url):\n    match = re.search(r\'https://([a-zA-Z0-9]+)\\.stackexchange\\.com\', url)\n    return match.group(1) if match else \'Unknown\'\n\n# Process the dataset\nfor item in dataset[\'train\']:  # Assuming we are processing the \'train\' split\n    # Extract topics from metadata\n    topics = [extract_topic(url) for url in item[\'metadata\'] if \'stackexchange.com\' in url]\n\n    # Use the first topic as the primary topic for this entry\n    primary_topic = topics[0] if topics else \'Unknown\'\n\n    # Add to processed data\n    processed_data.append({\n        "qid": item[\'qid\'],\n        "question": item[\'question\'],\n        "answers": item[\'answers\'],\n        "date": item[\'date\'],\n        "metadata": item[\'metadata\'],\n        "topic"

In [ ]:
# TODO: something for other than categorization: ex: is it negative text? is it optimist? is it against a certain subject of pro?
# clickbait, self promotion, bullying

In [ ]:
# https://github.com/causalNLP/logical-fallacy/blob/main/data/edu_all.csv
# logical fallacies dataset

# URL of the CSV file
url = "https://github.com/causalNLP/logical-fallacy/raw/main/data/edu_all.csv"

# Read the CSV file directly from the URL
data = pd.read_csv(url)

# Select only the required columns
fallacies_processed_data = data[['updated_label', 'source_article']].copy()

# Rename the columns to 'description' and 'text'
fallacies_processed_data.rename(columns={'updated_label': 'description', 'source_article': 'text'}, inplace=True)



In [ ]:
# https://huggingface.co/datasets/Open-Orca/OpenOrca?row=10
# for robustness could maybe have a description: "is an answer to this question"


In [ ]:
# ask gpt4 do generate answers that fit a description?

def get_texts_from_description(description):
  response = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a helpful assistant that generates snippets 1 per line"},
      {"role": "user", "content": f"Please generate a list of 5 text snippets that are exemples that fit this description: {description}."},
    ])

  text = response.choices[0].message.content
  texts = text.split("\n")
  return texts


In [ ]:
print(get_texts_from_description("Toxic online comments"))

['1. "You\'re obviously too stupid to understand this, why even bother trying?"', '2. "Only a moron would hold such an opinion, get a grip on reality."', '3. "People like you are what\'s wrong with this world. You should be ashamed for bringing such worthless views here."', '4. "Are you always this ignorant, or are you making a special effort today?"', '5. "Your ideas are laughable at best. I\'m embarrassed for you, how do you function with such a small brain?"']


In [ ]:

descriptions = [
    "Toxic online comments",
    "sentences that might be seen online and are overtly self-promotional",
    "sentences that use Overly Specific Jargon or Technical Language",
    "Sensationalist or Clickbait Titles",
    "Spoilers for Media",
    "Spam and advertizing"
]

gen_data = []
for description in descriptions:
  texts =  get_texts_from_description(description)
  for text in texts:
    gen_data.append({
        "description": description,
        "text": text
    })


In [ ]:

datasets = [
    [gen_data, "custom"],
    [fallacies_processed_data, "fallacies"],
    [wiki_processed_data, "wiki"],
    [yahoo_processed_data, "yahoo"],
    [tweet_processed_data, "tweet"]
    ]

dataframes = []
for dataset in datasets:
  df = pd.DataFrame(dataset[0])
  df['origin_dataset'] = dataset[1]
  dataframes.append(df)


# Concatenate all datasets
combined_dataset = pd.concat(dataframes, ignore_index=True)


In [ ]:
combined_dataset

,description,text,origin_dataset
0,Toxic online comments,"1. ""Your opinions are worth nothing. You clear...",custom
1,Toxic online comments,"2. ""If you can't understand this simple concep...",custom
2,Toxic online comments,"3. ""No one cares about your stupid posts. Do u...",custom
3,Toxic online comments,"4. ""Your lack of intelligence astounds me. How...",custom
4,Toxic online comments,"5. ""You're such a waste of time. Thanks for ru...",custom
...,...,...,...
12533,diaries_&_daily_life,SIGN THE PETITION. SOGN OTHER PETITIONS; Tony ...,tweet
12534,diaries_&_daily_life,The Democratic National Convention {{USERNAME}...,tweet
12535,diaries_&_daily_life,The most glorious morning and perfect day to s...,tweet
12536,diaries_&_daily_life,Sunday morning {@h@} concert youtube holes wit...,tweet


In [ ]:
from datasets import Dataset

df = combined_dataset
combined_dataset = df[df['text'].str.strip() != '']

hg_dataset = Dataset.from_pandas(combined_dataset, split='train')
hg_dataset.push_to_hub("Looyyd/description_text_pairs", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

In [ ]:

# create the model evaluation dataset
# this one will have the followign columns:
# text, descriptions, answers

import random

# Load your original dataset
# Replace this with your actual data loading method
original_df = combined_dataset

# Shuffle the DataFrame
shuffled_df = original_df.sample(frac=1).reset_index(drop=True)

# Split the DataFrame into two halves
midpoint = len(shuffled_df) // 2
first_half = shuffled_df.iloc[:midpoint]
second_half = shuffled_df.iloc[midpoint:]

# Initialize the list for the new dataset
new_data = []

# Function to add rows to new_data
def add_rows(df, include_original):
    for index, row in df.iterrows():
        original_text = row['text']
        original_description = row['description']

        # Create a pool of descriptions
        descriptions_pool = original_df['description'].tolist()

        # Randomly select descriptions (1 to 4)
        num_descriptions = random.randint(1, 4)
        selected_descriptions = random.sample(descriptions_pool, num_descriptions)

        if include_original:
            if original_description not in selected_descriptions:
                selected_descriptions[0] = original_description
            random.shuffle(selected_descriptions)  # Shuffle to randomize position of original description

        # Generate answers
        answers = [1 if desc == original_description else -1 for desc in selected_descriptions]

        # Create the new row
        new_row = {
            "text": original_text,
            "descriptions": selected_descriptions,
            "answers": answers
        }
        new_data.append(new_row)

# Add rows for both halves
add_rows(first_half, include_original=True)
add_rows(second_half, include_original=False)

# Convert the new data to a DataFrame
new_df = pd.DataFrame(new_data)

In [ ]:
hg_dataset_model = Dataset.from_pandas(new_df, split='train')
hg_dataset_model.push_to_hub("Looyyd/blissbrowse_classification", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
new_df

NameError: ignored